![Kayak][logo]

[logo]: https://play-lh.googleusercontent.com/qxPKLrBi-zwgDqsYf9FtbrHuxbkT_VVS1thV6F5bDw-WJvFiD_9LvTz0VejNFtMnmnE "To go kayaking."

# 서울시 공공 자전거 (따릉이) 고장 데이터 셋 


- 최종 데이터프레임 컬럼

| columns name | mean | datatype | count | non-null | 
|---|:---:|---:|---:|---:|
| `자전거번호` | 각 자전거 고유 번호 | `object` |  **20571** |**non-null** |
| `등록일시` | 고장신고 등록날짜 | `datetime64` |  **20571** |**non-null** |
| `등록일시2` | 대여이력 등록날짜 | `object`  |  **20571** |**non-null** |
| `고장구분` | 고장상태 구분 | `object` | **20571** |**non-null** |
| `총이용시간` | 대여일 이후 고장신고 날까지의 이용시간 | `float64` | **20571** |**non-null** |
| `총이용거리` | 대여일 이후 고장신고 날까지의 이용거리 | `float64` | **20571** |**non-null** |
| `날짜차이` | 고장신고 이후 재고장까지의 일수 |`int64`  | **20571** |**non-null** |
| `고장구분별_고장횟수` | 자전거, 고장부분 별 고장 횟수 | `int64` | **20571** |**non-null** |




In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from textwrap import wrap
from matplotlib.lines import Line2D
from matplotlib.offsetbox import OffsetImage, AnnotationBbox


## 샘플링 데이터 로드 및 결합

In [2]:
#21년 08월 데이터 로드
df_01 = pd.read_csv('df_break_21_sampled1.csv', encoding = 'cp949', low_memory = False)
#21년 09월 데이터 로드
df_02 = pd.read_csv('df_break_21_sampled2.csv', encoding = 'cp949', low_memory = False)
#08, 09월 데이터 열 병합
df_03 = pd.read_csv('df_break_21_sampled3.csv', encoding = 'cp949', low_memory = False)
df_04 = pd.read_csv('df_break_21_sampled4.csv', encoding = 'cp949', low_memory = False)
df_05 = pd.read_csv('df_break_21_sampled5.csv', encoding = 'cp949', low_memory = False)

df_new = pd.concat([df_01,df_02,df_03,df_04,df_05])

In [3]:
df_new

,자전거번호,등록일시,고장구분
0,SPB-32980,2021-01-02,타이어
1,SPB-30448,2021-01-02,타이어
2,SPB-30756,2021-01-02,기타
3,SPB-22059,2021-01-03,단말기
4,SPB-52225,2021-01-03,안장
...,...,...,...
13305,SPB-31161,2021-12-30,체인
13306,SPB-54054,2021-12-30,기타
13307,SPB-32054,2021-12-30,타이어
13308,SPB-39284,2021-12-30,안장


__info 함수를 사용하여 각 columns 별 dtype 확인__


In [4]:
df_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 67376 entries, 0 to 13309
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   자전거번호   67376 non-null  object
 1   등록일시    67376 non-null  object
 2   고장구분    67376 non-null  object
dtypes: object(3)
memory usage: 2.1+ MB


__메모리 절약을 위한 적재된 메모리 해제__

In [5]:
del df_01
del df_02
del df_03
del df_04
del df_05

In [6]:
# check dtype
df_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 67376 entries, 0 to 13309
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   자전거번호   67376 non-null  object
 1   등록일시    67376 non-null  object
 2   고장구분    67376 non-null  object
dtypes: object(3)
memory usage: 2.1+ MB


In [7]:
df_new.shape

(67376, 3)

## 고장(bike_breakdown) 데이터셋 불러오기

In [8]:
#고장 데이터셋 로드
df_break = pd.read_csv('bike_breakdown.csv', encoding = 'cp949')

# 고장(bike_breakdown) 데이터 전처리


## 자전거번호, 등록일시, 고장구분 총 3개의 컬럼 이용하여 정렬 'rank' 컬럼 생성

In [9]:
#df_break = df_break.sort_values(by=['등록일시'])
cols = ['자전거번호','고장구분','등록일시']


In [10]:
df_break = df_break.sort_values(by=['자전거번호'])

tups = df_break[cols].sort_values(by=cols, ascending=(True, True,True)).apply(tuple, axis=1)
f, i = pd.factorize(tups)
display(tups, f, i)

10160     (SPB-00001, 기타 , 2018-05-17)
12377     (SPB-00001, 기타 , 2018-06-18)
14514     (SPB-00001, 기타 , 2018-07-05)
14575     (SPB-00001, 기타 , 2018-07-06)
15481     (SPB-00001, 기타 , 2018-07-13)
                      ...             
336245    (SPB-84001, 기타 , 2021-12-23)
326098    (SPB-84018, 기타 , 2021-11-19)
327700    (SPB-84039, 단말기, 2021-11-24)
335361    (SPB-84149, 기타 , 2021-12-21)
316857     (SPB-84176, 페달, 2021-10-29)
Length: 337873, dtype: object

array([     0,      1,      2, ..., 334678, 334679, 334680], dtype=int64)

Index([ ('SPB-00001', '기타 ', '2018-05-17'),
        ('SPB-00001', '기타 ', '2018-06-18'),
        ('SPB-00001', '기타 ', '2018-07-05'),
        ('SPB-00001', '기타 ', '2018-07-06'),
        ('SPB-00001', '기타 ', '2018-07-13'),
        ('SPB-00001', '기타 ', '2018-07-17'),
        ('SPB-00001', '기타 ', '2018-08-10'),
        ('SPB-00001', '기타 ', '2018-08-30'),
        ('SPB-00001', '기타 ', '2018-09-08'),
        ('SPB-00001', '기타 ', '2019-01-04'),
       ...
         ('SPB-83840', '체인', '2021-12-01'),
        ('SPB-83864', '기타 ', '2021-12-21'),
        ('SPB-83904', '기타 ', '2021-11-15'),
        ('SPB-83917', '기타 ', '2021-12-21'),
       ('SPB-83956', '타이어 ', '2021-11-18'),
        ('SPB-84001', '기타 ', '2021-12-23'),
        ('SPB-84018', '기타 ', '2021-11-19'),
        ('SPB-84039', '단말기', '2021-11-24'),
        ('SPB-84149', '기타 ', '2021-12-21'),
         ('SPB-84176', '페달', '2021-10-29')],
      dtype='object', length=334681)

In [11]:
factorized = pd.Series(f + 1, tups.index)
display(factorized)

10160          1
12377          2
14514          3
14575          4
15481          5
           ...  
336245    334677
326098    334678
327700    334679
335361    334680
316857    334681
Length: 337873, dtype: int64

In [12]:
df_break['rank'] = factorized
df_break.sort_values(by='rank', ascending=True, inplace = True)

display(df_break)

,자전거번호,등록일시,고장구분,rank
10160,SPB-00001,2018-05-17,기타,1
12377,SPB-00001,2018-06-18,기타,2
14514,SPB-00001,2018-07-05,기타,3
14575,SPB-00001,2018-07-06,기타,4
15481,SPB-00001,2018-07-13,기타,5
...,...,...,...,...
336245,SPB-84001,2021-12-23,기타,334677
326098,SPB-84018,2021-11-19,기타,334678
327700,SPB-84039,2021-11-24,단말기,334679
335361,SPB-84149,2021-12-21,기타,334680


__rank로 오름차순 정렬하면 성공__

In [13]:
df_break.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 337873 entries, 10160 to 316857
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   자전거번호   337873 non-null  object
 1   등록일시    337873 non-null  object
 2   고장구분    337873 non-null  object
 3   rank    337873 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 12.9+ MB


__등록일시 컬럼 타입변환 (object to datetime)__

In [14]:
df_break['등록일시'] = pd.to_datetime(df_break['등록일시'], format="%Y/%m/%d")


In [15]:
#결측치 확인
df_break['등록일시'].isna().sum()

0

In [16]:
df_break.groupby(['자전거번호'])

In [17]:
df_break['자전거번호']

10160     SPB-00001
12377     SPB-00001
14514     SPB-00001
14575     SPB-00001
15481     SPB-00001
            ...    
336245    SPB-84001
326098    SPB-84018
327700    SPB-84039
335361    SPB-84149
316857    SPB-84176
Name: 자전거번호, Length: 337873, dtype: object

In [18]:
df_break.head()

,자전거번호,등록일시,고장구분,rank
10160,SPB-00001,2018-05-17,기타,1
12377,SPB-00001,2018-06-18,기타,2
14514,SPB-00001,2018-07-05,기타,3
14575,SPB-00001,2018-07-06,기타,4
15481,SPB-00001,2018-07-13,기타,5


In [19]:
df_break.tail()

,자전거번호,등록일시,고장구분,rank
336245,SPB-84001,2021-12-23,기타,334677
326098,SPB-84018,2021-11-19,기타,334678
327700,SPB-84039,2021-11-24,단말기,334679
335361,SPB-84149,2021-12-21,기타,334680
316857,SPB-84176,2021-10-29,페달,334681


__- 총 고장 컬럼 전체를 1로 초기화 후 df_break & df_new의 dataframe key값 일치작업__

In [20]:

df_break['총_고장'] = 1

In [21]:
df_break['자전거번호'] = df_break['자전거번호'].str.split('-').str[1]
df_new['자전거번호'] = df_new['자전거번호'].str.split('-').str[1]

In [22]:
#t:m:s 날리기 위한 split 작업 후 대입
df_break['등록일시'] = df_break['등록일시'].dt.strftime('%Y/%m/%d')
df_break['등록일시']

10160     2018/05/17
12377     2018/06/18
14514     2018/07/05
14575     2018/07/06
15481     2018/07/13
             ...    
336245    2021/12/23
326098    2021/11/19
327700    2021/11/24
335361    2021/12/21
316857    2021/10/29
Name: 등록일시, Length: 337873, dtype: object

In [23]:
df_break

,자전거번호,등록일시,고장구분,rank,총_고장
10160,00001,2018/05/17,기타,1,1
12377,00001,2018/06/18,기타,2,1
14514,00001,2018/07/05,기타,3,1
14575,00001,2018/07/06,기타,4,1
15481,00001,2018/07/13,기타,5,1
...,...,...,...,...,...
336245,84001,2021/12/23,기타,334677,1
326098,84018,2021/11/19,기타,334678,1
327700,84039,2021/11/24,단말기,334679,1
335361,84149,2021/12/21,기타,334680,1


## 데이터프레임 결측치 확인 및 제거

In [24]:
df_break = df_break.dropna()

In [25]:
df_break

,자전거번호,등록일시,고장구분,rank,총_고장
10160,00001,2018/05/17,기타,1,1
12377,00001,2018/06/18,기타,2,1
14514,00001,2018/07/05,기타,3,1
14575,00001,2018/07/06,기타,4,1
15481,00001,2018/07/13,기타,5,1
...,...,...,...,...,...
336245,84001,2021/12/23,기타,334677,1
326098,84018,2021/11/19,기타,334678,1
327700,84039,2021/11/24,단말기,334679,1
335361,84149,2021/12/21,기타,334680,1


__group_col list에 있는 조건별로 그룹화 뒤, 연산__

In [26]:

group_col =  ['자전거번호','등록일시','고장구분']
df_group = df_break.groupby(group_col).sum()


In [27]:

df_group.head(50)



rank  총_고장
자전거번호 등록일시       고장구분            
00001 2018/05/17 기타       1     1
      2018/05/28 단말기     44     1
                 페달     173     1
      2018/06/18 기타       2     1
                 단말기     45     1
                 안장     103     1
                 체인     118     1
                 타이어    146     1
                 페달     174     1
      2018/06/22 타이어    147     1
      2018/06/23 페달     175     1
      2018/06/28 안장     104     1
                 타이어    148     1
      2018/07/04 단말기     46     1
      2018/07/05 기타       3     1
                 체인     119     1
      2018/07/06 기타       4     1
                 타이어    149     1
      2018/07/08 타이어    150     1
      2018/07/13 기타       5     1
      2018/07/14 체인     120     1
                 페달     176     1
      2018/07/17 기타       6     1
      2018/07/21 체인     121     1
      2018/07/27 타이어    151     1
      2018/08/03 단말기     47     1
      2018/08/10 기타       7     1
      2018/08/13 단말기     48     1
                 타이어    304     2
      2018/08/15 단말기     49     1
      2018/08/24 단말기     50     1
      2018/08/30 기타       8     1
                 단말기     51     1
      2018/09/04 단말기     52     1
      2018/09/05 타이어    153     1
      2018/09/07 체인     122     1
      2018/09/08 기타       9     1
      2018/09/10 단말기     53     1
      2018/09/11 단말기     54     1
      2018/09/12 타이어    154     1
      2018/09/28 타이어    155     1
      2018/10/01 안장     105     1
      2018/10/03 안장     106     1
      2018/10/09 단말기     55     1
      2018/10/20 안장     107     1
      2018/10/29 체인     123     1
      2018/11/04 체인     124     1
                 타이어    156     1
      2018/11/06 단말기     56     1
      2018/11/12 단말기     57     1

In [28]:

df_group.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 334681 entries, ('00001', '2018/05/17', '기타 ') to ('84176', '2021/10/29', '페달')
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype
---  ------  --------------   -----
 0   rank    334681 non-null  int64
 1   총_고장    334681 non-null  int64
dtypes: int64(2)
memory usage: 7.7+ MB


In [29]:
df_group_reset = df_group.reset_index()

In [30]:
df_group_reset.head()

,자전거번호,등록일시,고장구분,rank,총_고장
0,00001,2018/05/17,기타,1,1
1,00001,2018/05/28,단말기,44,1
2,00001,2018/05/28,페달,173,1
3,00001,2018/06/18,기타,2,1
4,00001,2018/06/18,단말기,45,1


In [31]:
df_group_reset.tail()

,자전거번호,등록일시,고장구분,rank,총_고장
334676,84001,2021/12/23,기타,334677,1
334677,84018,2021/11/19,기타,334678,1
334678,84039,2021/11/24,단말기,334679,1
334679,84149,2021/12/21,기타,334680,1
334680,84176,2021/10/29,페달,334681,1


In [32]:
df_group_new = df_group
df_group_new

rank  총_고장
자전거번호 등록일시       고장구분              
00001 2018/05/17 기타         1     1
      2018/05/28 단말기       44     1
                 페달       173     1
      2018/06/18 기타         2     1
                 단말기       45     1
...                       ...   ...
84001 2021/12/23 기타    334677     1
84018 2021/11/19 기타    334678     1
84039 2021/11/24 단말기   334679     1
84149 2021/12/21 기타    334680     1
84176 2021/10/29 페달    334681     1

[334681 rows x 2 columns]

## 고장신고 이후 재고장 신고일까지의 날짜 계산

In [33]:
df_break_heetaek = df_break.assign(등록일시2=df_break.groupby(['자전거번호','고장구분']).등록일시.shift(-1)).fillna({'등록일시2': df_break.등록일시})


In [34]:
df_break_heetaek

,자전거번호,등록일시,고장구분,rank,총_고장,등록일시2
10160,00001,2018/05/17,기타,1,1,2018/06/18
12377,00001,2018/06/18,기타,2,1,2018/07/05
14514,00001,2018/07/05,기타,3,1,2018/07/06
14575,00001,2018/07/06,기타,4,1,2018/07/13
15481,00001,2018/07/13,기타,5,1,2018/07/17
...,...,...,...,...,...,...
336245,84001,2021/12/23,기타,334677,1,2021/12/23
326098,84018,2021/11/19,기타,334678,1,2021/11/19
327700,84039,2021/11/24,단말기,334679,1,2021/11/24
335361,84149,2021/12/21,기타,334680,1,2021/12/21


In [35]:
df_break_heetaek.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 337873 entries, 10160 to 316857
Data columns (total 6 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   자전거번호   337873 non-null  object
 1   등록일시    337873 non-null  object
 2   고장구분    337873 non-null  object
 3   rank    337873 non-null  int64 
 4   총_고장    337873 non-null  int64 
 5   등록일시2   337873 non-null  object
dtypes: int64(2), object(4)
memory usage: 18.0+ MB


__datetime format 변환__

In [36]:
df_break_heetaek['등록일시'] = pd.to_datetime(df_break_heetaek['등록일시'], format="%Y/%m/%d")
df_break_heetaek['등록일시2'] = pd.to_datetime(df_break_heetaek['등록일시2'], format="%Y/%m/%d")
df_break_heetaek


,자전거번호,등록일시,고장구분,rank,총_고장,등록일시2
10160,00001,2018-05-17,기타,1,1,2018-06-18
12377,00001,2018-06-18,기타,2,1,2018-07-05
14514,00001,2018-07-05,기타,3,1,2018-07-06
14575,00001,2018-07-06,기타,4,1,2018-07-13
15481,00001,2018-07-13,기타,5,1,2018-07-17
...,...,...,...,...,...,...
336245,84001,2021-12-23,기타,334677,1,2021-12-23
326098,84018,2021-11-19,기타,334678,1,2021-11-19
327700,84039,2021-11-24,단말기,334679,1,2021-11-24
335361,84149,2021-12-21,기타,334680,1,2021-12-21


__차이값 발생__

In [37]:
df_break_heetaek['heetaek_diff'] = df_break_heetaek['등록일시2'] - df_break_heetaek['등록일시']

In [38]:
df_break_heetaek.tail(50)

,자전거번호,등록일시,고장구분,rank,총_고장,등록일시2,heetaek_diff
322304,83002,2021-11-10,기타,334632,1,2021-11-10,0 days
321379,83004,2021-11-06,기타,334633,1,2021-11-06,0 days
334108,83017,2021-12-15,타이어,334634,1,2021-12-15,0 days
326853,83056,2021-11-21,타이어,334635,1,2021-11-21,0 days
324902,83071,2021-11-16,단말기,334636,1,2021-11-16,0 days
324020,83073,2021-11-14,기타,334637,1,2021-11-14,0 days
337257,83075,2021-12-28,기타,334638,1,2021-12-28,0 days
328314,83079,2021-11-25,체인,334639,1,2021-11-25,0 days
328313,83079,2021-11-25,타이어,334640,1,2021-11-25,0 days
326527,83100,2021-11-20,기타,334641,1,2021-11-20,0 days


__diff값 split 적용을 위해 str로 타입변경__


In [39]:
df_break_heetaek['heetaek_diff'] = df_break_heetaek['heetaek_diff'].astype('str')

In [40]:
df_break_heetaek.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 337873 entries, 10160 to 316857
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   자전거번호         337873 non-null  object        
 1   등록일시          337873 non-null  datetime64[ns]
 2   고장구분          337873 non-null  object        
 3   rank          337873 non-null  int64         
 4   총_고장          337873 non-null  int64         
 5   등록일시2         337873 non-null  datetime64[ns]
 6   heetaek_diff  337873 non-null  object        
dtypes: datetime64[ns](2), int64(2), object(3)
memory usage: 20.6+ MB


__split 실행 후, int형 변환__

In [41]:
df_break_heetaek['heetaek_diff'] = df_break_heetaek['heetaek_diff'].str.split(' ').str[0]
df_break_heetaek['heetaek_diff'] = df_break_heetaek['heetaek_diff'].astype('int64')


In [42]:
df_break_heetaek.head(50)

,자전거번호,등록일시,고장구분,rank,총_고장,등록일시2,heetaek_diff
10160,00001,2018-05-17,기타,1,1,2018-06-18,32
12377,00001,2018-06-18,기타,2,1,2018-07-05,17
14514,00001,2018-07-05,기타,3,1,2018-07-06,1
14575,00001,2018-07-06,기타,4,1,2018-07-13,7
15481,00001,2018-07-13,기타,5,1,2018-07-17,4
16149,00001,2018-07-17,기타,6,1,2018-08-10,24
19284,00001,2018-08-10,기타,7,1,2018-08-30,20
21698,00001,2018-08-30,기타,8,1,2018-09-08,9
23315,00001,2018-09-08,기타,9,1,2019-01-04,118
36868,00001,2019-01-04,기타,10,1,2019-01-23,19


In [43]:
df_break = df_break_heetaek

df_break.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 337873 entries, 10160 to 316857
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   자전거번호         337873 non-null  object        
 1   등록일시          337873 non-null  datetime64[ns]
 2   고장구분          337873 non-null  object        
 3   rank          337873 non-null  int64         
 4   총_고장          337873 non-null  int64         
 5   등록일시2         337873 non-null  datetime64[ns]
 6   heetaek_diff  337873 non-null  int64         
dtypes: datetime64[ns](2), int64(3), object(2)
memory usage: 20.6+ MB


In [44]:
df_break = df_break[['자전거번호','등록일시','고장구분','총_고장','heetaek_diff']]

In [45]:
df_break.head()

,자전거번호,등록일시,고장구분,총_고장,heetaek_diff
10160,00001,2018-05-17,기타,1,32
12377,00001,2018-06-18,기타,1,17
14514,00001,2018-07-05,기타,1,1
14575,00001,2018-07-06,기타,1,7
15481,00001,2018-07-13,기타,1,4


In [46]:
df_break.tail()

,자전거번호,등록일시,고장구분,총_고장,heetaek_diff
336245,84001,2021-12-23,기타,1,0
326098,84018,2021-11-19,기타,1,0
327700,84039,2021-11-24,단말기,1,0
335361,84149,2021-12-21,기타,1,0
316857,84176,2021-10-29,페달,1,0


In [47]:
bike_number_list = []
bike_number_list = df_break['자전거번호'].unique()
bike_number_list = bike_number_list.tolist()

In [48]:
len(bike_number_list)

50764

In [49]:
cols_break = ['자전거번호','고장구분']

## 기간과_기관사이 총 고장횟수 누적값 계산

In [50]:
df_break['기간과_기관사이_총_고장횟수'] = df_break.groupby(cols_break)['총_고장'].apply(lambda x: x.cumsum())

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [51]:
df_break.head(50)

,자전거번호,등록일시,고장구분,총_고장,heetaek_diff,기간과_기관사이_총_고장횟수
10160,00001,2018-05-17,기타,1,32,1
12377,00001,2018-06-18,기타,1,17,2
14514,00001,2018-07-05,기타,1,1,3
14575,00001,2018-07-06,기타,1,7,4
15481,00001,2018-07-13,기타,1,4,5
16149,00001,2018-07-17,기타,1,24,6
19284,00001,2018-08-10,기타,1,20,7
21698,00001,2018-08-30,기타,1,9,8
23315,00001,2018-09-08,기타,1,118,9
36868,00001,2019-01-04,기타,1,19,10


__df_group은 각 고장별 누적합을 위한 dataframe__

In [52]:
df_group = df_break.groupby(cols_break).max()
df_group.reset_index()

,자전거번호,고장구분,등록일시,총_고장,heetaek_diff,기간과_기관사이_총_고장횟수
0,00001,기타,2021-10-28,1,605,45
1,00001,단말기,2021-10-28,1,754,65
2,00001,안장,2019-10-05,1,165,15
3,00001,체인,2019-10-09,1,124,28
4,00001,타이어,2019-10-22,1,89,28
...,...,...,...,...,...,...
164832,84001,기타,2021-12-23,1,0,1
164833,84018,기타,2021-11-19,1,0,1
164834,84039,단말기,2021-11-24,1,0,1
164835,84149,기타,2021-12-21,1,0,1


In [53]:
df_group_test = df_group.reset_index()


In [54]:
df_group_test['고장구분'].unique()

array(['기타 ', '단말기', '안장', '체인', '타이어 ', '페달', '파손', '잠금장치 불량'],
      dtype=object)

In [55]:
df_group_test

,자전거번호,고장구분,등록일시,총_고장,heetaek_diff,기간과_기관사이_총_고장횟수
0,00001,기타,2021-10-28,1,605,45
1,00001,단말기,2021-10-28,1,754,65
2,00001,안장,2019-10-05,1,165,15
3,00001,체인,2019-10-09,1,124,28
4,00001,타이어,2019-10-22,1,89,28
...,...,...,...,...,...,...
164832,84001,기타,2021-12-23,1,0,1
164833,84018,기타,2021-11-19,1,0,1
164834,84039,단말기,2021-11-24,1,0,1
164835,84149,기타,2021-12-21,1,0,1


In [56]:
df_group = df_group.reset_index()

## 데이터프레임 Join 후 Preprocessing

In [57]:
df_merge = pd.merge(df_break,df_group, how = 'inner', on = (cols_break))

In [58]:
df_merge

,자전거번호,등록일시_x,고장구분,총_고장_x,heetaek_diff_x,기간과_기관사이_총_고장횟수_x,등록일시_y,총_고장_y,heetaek_diff_y,기간과_기관사이_총_고장횟수_y
0,00001,2018-05-17,기타,1,32,1,2021-10-28,1,605,45
1,00001,2018-06-18,기타,1,17,2,2021-10-28,1,605,45
2,00001,2018-07-05,기타,1,1,3,2021-10-28,1,605,45
3,00001,2018-07-06,기타,1,7,4,2021-10-28,1,605,45
4,00001,2018-07-13,기타,1,4,5,2021-10-28,1,605,45
...,...,...,...,...,...,...,...,...,...,...
337868,84001,2021-12-23,기타,1,0,1,2021-12-23,1,0,1
337869,84018,2021-11-19,기타,1,0,1,2021-11-19,1,0,1
337870,84039,2021-11-24,단말기,1,0,1,2021-11-24,1,0,1
337871,84149,2021-12-21,기타,1,0,1,2021-12-21,1,0,1


In [59]:
df_merge = df_merge[['등록일시_x','자전거번호','고장구분','heetaek_diff_x','기간과_기관사이_총_고장횟수_y']]

In [60]:
df_merge.sample(50)

,등록일시_x,자전거번호,고장구분,heetaek_diff_x,기간과_기관사이_총_고장횟수_y
110813,2019-04-17,17741,단말기,0,3
302787,2021-09-24,50565,체인,0,5
169312,2021-04-22,31157,페달,26,5
256635,2021-07-26,41836,체인,95,2
167572,2021-08-28,30997,타이어,0,3
189467,2021-10-06,32920,페달,0,1
176781,2020-11-05,31857,기타,269,3
253904,2020-08-24,41477,기타,210,4
175076,2021-09-26,31715,안장,17,4
101615,2018-07-21,16536,체인,104,2


In [61]:

df_merge = df_merge.rename(columns={'등록일시_x':'등록일시'})
df_merge = df_merge.rename(columns={'heetaek_diff_x':'날짜차이'})
df_merge = df_merge.rename(columns={'기간과_기관사이_총_고장횟수_y':'기간과_기관사이_총_고장횟수'})

In [62]:
group_col =  ['자전거번호','등록일시','고장구분']
df_group = df_break.groupby(group_col).max()
df_group.reset_index()

,자전거번호,등록일시,고장구분,총_고장,heetaek_diff,기간과_기관사이_총_고장횟수
0,00001,2018-05-17,기타,1,32,1
1,00001,2018-05-28,단말기,1,21,1
2,00001,2018-05-28,페달,1,21,1
3,00001,2018-06-18,기타,1,17,2
4,00001,2018-06-18,단말기,1,16,2
...,...,...,...,...,...,...
334676,84001,2021-12-23,기타,1,0,1
334677,84018,2021-11-19,기타,1,0,1
334678,84039,2021-11-24,단말기,1,0,1
334679,84149,2021-12-21,기타,1,0,1


In [63]:
df_merge.head(50)

,등록일시,자전거번호,고장구분,날짜차이,기간과_기관사이_총_고장횟수
0,2018-05-17,00001,기타,32,45
1,2018-06-18,00001,기타,17,45
2,2018-07-05,00001,기타,1,45
3,2018-07-06,00001,기타,7,45
4,2018-07-13,00001,기타,4,45
5,2018-07-17,00001,기타,24,45
6,2018-08-10,00001,기타,20,45
7,2018-08-30,00001,기타,9,45
8,2018-09-08,00001,기타,118,45
9,2019-01-04,00001,기타,19,45


In [64]:
#df_break.to_csv("df_break2.csv", mode='w',encoding='euc-kr')

df_merge = df_merge.rename(columns={'기간과_기관사이_총_고장횟수':'고장구분별_고장횟수'})

## 대여 이력 데이터 로드 및 정제 작업

In [65]:
df_refined = pd.read_csv('df_refined2.csv', encoding = 'cp949', low_memory = False)


In [66]:
df_refined

,자전거번호,등록일시,등록일시2,고장구분,총이용시간,총이용거리
0,SPB-40402,2021-01-01,2021-07-13,기타,5972.0,711506.91
1,SPB-32280,2021-01-01,2021-08-12,체인,3236.0,425409.52
2,SPB-34462,2021-01-02,2021-06-10,기타,8221.0,956419.86
3,SPB-35692,2021-01-02,2021-05-19,단말기,3503.0,439067.44
4,SPB-53177,2021-01-02,2021-05-11,체인,1250.0,109193.77
...,...,...,...,...,...,...
32285,SPB-54238,2021-12-28,2021-12-30,기타,125.0,11390.00
32286,SPB-33841,2021-12-28,2021-12-29,체인,6.0,88.17
32287,SPB-48213,2021-12-28,2021-12-30,안장,144.0,12570.00
32288,SPB-37000,2021-12-28,2021-12-30,타이어,2.0,141.90


In [67]:
df_refined['자전거번호'] = df_refined['자전거번호'].str.split('-').str[1]

In [68]:
condition_list = ['자전거번호','등록일시','등록일시2','고장구분']

In [69]:
df_refined_reset = df_refined.groupby(condition_list).sum()

In [70]:
df_refined_reset = df_refined_reset.reset_index()
df_refined_reset


,자전거번호,등록일시,등록일시2,고장구분,총이용시간,총이용거리
0,23825,2021-04-29,2021-05-06,단말기,61.0,8750.00
1,30001,2021-08-24,2021-10-09,타이어,3206.0,353917.70
2,30001,2021-09-14,2021-11-17,체인,4382.0,462030.56
3,30008,2021-04-29,2021-06-13,체인,9402.0,1160233.26
4,30008,2021-05-18,2021-06-16,기타,712.0,83736.18
...,...,...,...,...,...,...
26716,81837,2021-07-23,2021-09-20,기타,2005.0,205930.00
26717,81918,2021-07-22,2021-08-08,기타,560.0,68230.00
26718,81981,2021-08-24,2021-09-15,기타,3812.0,213520.00
26719,81988,2021-08-10,2021-09-12,단말기,1868.0,182920.00


In [71]:
df_refined_reset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26721 entries, 0 to 26720
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   자전거번호   26721 non-null  object 
 1   등록일시    26721 non-null  object 
 2   등록일시2   26721 non-null  object 
 3   고장구분    26721 non-null  object 
 4   총이용시간   26721 non-null  float64
 5   총이용거리   26721 non-null  float64
dtypes: float64(2), object(4)
memory usage: 1.2+ MB


In [72]:
df_merge.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 337873 entries, 0 to 337872
Data columns (total 5 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   등록일시        337873 non-null  datetime64[ns]
 1   자전거번호       337873 non-null  object        
 2   고장구분        337873 non-null  object        
 3   날짜차이        337873 non-null  int64         
 4   고장구분별_고장횟수  337873 non-null  int64         
dtypes: datetime64[ns](1), int64(2), object(2)
memory usage: 15.5+ MB


In [73]:
df_refined_reset['등록일시'] = pd.to_datetime(df_refined_reset['등록일시'], format="%Y/%m/%d")


# 최종 데이터 프레임 생성

## 대여이력 및 고장이력 데이터프레임 병합

In [74]:
cols2 = ['자전거번호','등록일시','고장구분']

In [75]:
df_final = pd.merge(df_refined_reset,df_merge, how = 'inner', on = cols2)

In [76]:
df_final.head(50)

,자전거번호,등록일시,등록일시2,고장구분,총이용시간,총이용거리,날짜차이,고장구분별_고장횟수
0,23825,2021-04-29,2021-05-06,단말기,61.0,8750.00,7,3
1,30001,2021-08-24,2021-10-09,타이어,3206.0,353917.70,46,4
2,30001,2021-09-14,2021-11-17,체인,4382.0,462030.56,64,2
3,30008,2021-04-29,2021-06-13,체인,9402.0,1160233.26,45,9
4,30008,2021-05-18,2021-06-16,기타,712.0,83736.18,29,8
5,30008,2021-06-13,2021-07-14,체인,2922.0,325563.42,31,9
6,30008,2021-06-16,2021-07-21,기타,2248.0,254247.74,35,8
7,30008,2021-07-14,2021-07-28,체인,1528.0,154152.72,14,9
8,30008,2021-07-21,2021-08-04,기타,1076.0,122762.64,14,8
9,30008,2021-07-28,2021-08-04,체인,1076.0,122762.64,7,9


In [77]:
df_final.tail(50)

,자전거번호,등록일시,등록일시2,고장구분,총이용시간,총이용거리,날짜차이,고장구분별_고장횟수
27125,80311,2021-08-07,2021-08-26,기타,258.0,37240.0,19,3
27126,80375,2021-04-09,2021-04-25,페달,772.0,71090.0,16,3
27127,80375,2021-04-25,2021-04-30,페달,28.0,2540.0,5,3
27128,80376,2021-09-25,2021-09-28,기타,28.0,2280.0,3,2
27129,80381,2021-12-10,2021-12-23,기타,87.0,10370.0,13,2
27130,80392,2021-01-15,2021-10-15,기타,8.0,1160.0,273,2
27131,80393,2021-05-07,2021-09-30,기타,6271.0,614620.0,146,2
27132,80393,2021-05-07,2021-10-10,단말기,443.0,49590.0,156,2
27133,80425,2021-04-19,2021-07-07,기타,5618.0,580610.0,79,2
27134,80462,2021-06-07,2021-06-20,기타,3008.0,335560.0,13,3


# 

In [78]:
df_final

,자전거번호,등록일시,등록일시2,고장구분,총이용시간,총이용거리,날짜차이,고장구분별_고장횟수
0,23825,2021-04-29,2021-05-06,단말기,61.0,8750.00,7,3
1,30001,2021-08-24,2021-10-09,타이어,3206.0,353917.70,46,4
2,30001,2021-09-14,2021-11-17,체인,4382.0,462030.56,64,2
3,30008,2021-04-29,2021-06-13,체인,9402.0,1160233.26,45,9
4,30008,2021-05-18,2021-06-16,기타,712.0,83736.18,29,8
...,...,...,...,...,...,...,...,...
27170,81837,2021-07-23,2021-09-20,기타,2005.0,205930.00,59,3
27171,81918,2021-07-22,2021-08-08,기타,560.0,68230.00,17,2
27172,81981,2021-08-24,2021-09-15,기타,3812.0,213520.00,22,2
27173,81988,2021-08-10,2021-09-12,단말기,1868.0,182920.00,33,2


## 결측치 최종 확인

In [79]:
df_final.isnull().sum()

자전거번호         0
등록일시          0
등록일시2         0
고장구분          0
총이용시간         0
총이용거리         0
날짜차이          0
고장구분별_고장횟수    0
dtype: int64

In [80]:
df_final.to_csv("df_final.csv", mode='w',encoding='cp949')


In [81]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27175 entries, 0 to 27174
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   자전거번호       27175 non-null  object        
 1   등록일시        27175 non-null  datetime64[ns]
 2   등록일시2       27175 non-null  object        
 3   고장구분        27175 non-null  object        
 4   총이용시간       27175 non-null  float64       
 5   총이용거리       27175 non-null  float64       
 6   날짜차이        27175 non-null  int64         
 7   고장구분별_고장횟수  27175 non-null  int64         
dtypes: datetime64[ns](1), float64(2), int64(2), object(3)
memory usage: 1.9+ MB
